# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже пользуются тирифами Smart и Ultra.
Задача: построить модель для задачи *классификации*, которая выберет подходящий тариф. Предобработка данных уже сделана ранее.

Построить модель с максимально большим значением *accuracy*. Нелбходимо довести долю правильных ответов по крайней мере до 0.75. Проверить *accuracy* на тестовой выборке.

## Откройте и изучите файл

In [1]:
# импортируем нужные библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
from tqdm import tqdm
from sklearn.dummy import DummyClassifier

Читаем полученые данные:

In [2]:
# читаем данные спомощью try..except и read_csv
try:
    dt= pd.read_csv('/datasets/users_behavior.csv')
except:
    dt = pd.read_csv('/Users/refresh/Downloads/users_behavior.csv')
dt.info()
dt.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Описание данных:
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

- в таблице нет пропущеных значений
- используется нижний регистр
- надо поменять тип данных на целочисленный в количестве звонков и сообщениях

In [3]:
# меняем типы данных и проверяем
dt['calls'] = dt['calls'].astype('Int64', errors='ignore')
dt['messages'] = dt['messages'].astype('Int64', errors='ignore')
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   Int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   Int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: Int64(2), float64(2), int64(1)
memory usage: 131.9 KB


In [4]:
dt['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

В тарифах преобладает Smart.

## Разбейте данные на выборки

In [5]:
# формируем признаки и целевой признак
features = dt.drop('is_ultra', axis=1)
target = dt['is_ultra']
# выделяем тренировочную выборку и временную выборку 40%
features_train, features_40, target_train, target_40 = train_test_split(features, target,\
                                                            test_size=.40, random_state=23456, stratify=target)
# выделяем валидационную и тестовую выборки
features_valid, features_test, target_valid, target_test = train_test_split(features_40, target_40,\
                                                            test_size=.50, random_state=23456, stratify=target_40)
# проверка
print('начальный размер')
print(features.shape)
print(target.shape)
print('размер тренировочной выборки (60%)')
print(features_train.shape)
print(target_train.shape)
print('размер валидационной выборки (20%)')
print(features_valid.shape)
print(target_valid.shape)
print('размер тестовой выборки (20%)')
print(features_test.shape)
print(target_test.shape)

начальный размер
(3214, 4)
(3214,)
размер тренировочной выборки (60%)
(1928, 4)
(1928,)
размер валидационной выборки (20%)
(643, 4)
(643,)
размер тестовой выборки (20%)
(643, 4)
(643,)


## Исследуйте модели

### Решающее дерево

In [6]:
min_samples_leaf = [1, 2, 4, 6, 8]
min_samples_split = [2, 4, 6, 10]
# создадим переменные для сохранения лучшей модели, лучшей глубины, и лучшей доли правильных ответов
best_tree = None
best_decision_depth = 0
best_decision_split = 0
best_decision_leaf = 0
best_result_tree = 0
# цикл по глубине дерева
for depth in range(1,11):
    for split in [2, 4, 6, 10]:
        for leaf in [1, 2, 4, 6]:
            # создаём модель
            model_tree = DecisionTreeClassifier(random_state=23456, max_depth=depth,\
                                                min_samples_split=split, min_samples_leaf=leaf)
            # обучаем модель
            model_tree.fit(features_train, target_train)
            # считаем accuracy
            result_tree = model_tree.score(features_valid, target_valid)
            # условие для выбора лучшего accuracy и сохранение лучших показателей
            if result_tree > best_result_tree:
                best_tree = model_tree
                best_decision_depth = depth
                best_result_tree = result_tree
                best_decision_split = split
                best_decision_leaf = leaf
# вывод результата
print(f'лучшее дерево решений: глубина - {best_decision_depth}, accuracy - {best_result_tree}')
print(f'минимальное количество примеров для разделения - {best_decision_split}')
print(f'минимальное количество объектов в листе - {best_decision_leaf}')

лучшее дерево решений: глубина - 8, accuracy - 0.8055987558320373
минимальное количество примеров для разделения - 2
минимальное количество объектов в листе - 1


Из 160 деревьев решений лучший результат показало дерево с глубиной 8. Доля правильных ответов на валидационной выборке 0,805. Минимальное количество примеров для разделения - 2. Минимальное количество объектов в листе - 1

### Случайный лес

In [7]:
%%time 
# создадим переменные для сохранения лучшей модели, лучшего количества деревьев, и лучшей доли правильных ответов
best_forest = None
best_estimators = 0
best_forest_depth = 0
best_forest_split = 0
best_forest_leaf = 0
best_result_forest = 0
pp=0
# цикл по количеству деревьев
for est in tqdm(range(10, 51, 10)):
    for depth in range(1,11):
        for split in [2, 4, 6, 10]:
            for leaf in [1, 2, 4, 6]:
                pp+=1
                # создаём модель
                model_forest = RandomForestClassifier(random_state=23456, n_estimators=est, max_depth=depth\
                                                     , min_samples_split=split, min_samples_leaf=leaf)
                # обучаем модель
                model_forest.fit(features_train, target_train)
                # считаем accuracy
                result_forest = model_forest.score(features_valid, target_valid)
                # условие для выбора лучшего accuracy и сохранение лучших показателей
                if result_forest > best_result_forest:
                    best_forest = model_forest
                    best_estimators = est
                    best_result_forest = result_forest
                    best_forest_depth = depth
                    best_forest_split = split
                    best_forest_leaf = leaf
# вывод результата
print(f'лучший лес: количество деревьев - {best_estimators}, глубина дерева {best_forest_depth}, accuracy - {best_result_forest}')
print(f'минимальное количество примеров для разделения - {best_forest_split}')
print(f'минимальное количество объектов в листе - {best_forest_leaf}')

100%|██████████| 5/5 [00:56<00:00, 11.38s/it]

лучший лес: количество деревьев - 30, глубина дерева 8, accuracy - 0.8149300155520995
минимальное количество примеров для разделения - 2
минимальное количество объектов в листе - 2
CPU times: user 56.4 s, sys: 481 ms, total: 56.9 s
Wall time: 56.9 s


В случайном лесе мы рассмотрели 800 раз модель с разными параметрами, лучшая стала модель с 30ю деревьями и глубиной 8, доля правильных ответов на валидационной выборке 0,814. Минимальное количество примеров для разделения - 2. Минимальное количество объектов в листе - 2. Способ получился очень долгий и затратный, но обеспечил хорошую долю правильных ответов.

### Логистическая регрессия

In [8]:
# отключаем предупреждения
warnings.filterwarnings('ignore')
best_algorithm = ''
best_result_logistic = 0
for i in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
    # создаём модель
    model_logistic = LogisticRegression(random_state=23456, solver=i, max_iter=1000)
    # обучаем модель
    model_logistic.fit(features_train, target_train)
    # считаем accuracy
    result_logistic = model_logistic.score(features_valid, target_valid)
    if result_logistic > best_result_logistic:
        best_algorithm = i
        best_result_logistic = result_logistic
    # вывод результата
print(f'логистическая регрессия алгоритм {best_algorithm}: accuracy - {best_result_logistic}')

логистическая регрессия алгоритм newton-cg: accuracy - 0.7558320373250389


Логистическая регрессия с алгоритмом 'newton-cg' показала лучше остальных, доля правильных ответов на валидационной выборке 0,755.

## Проверьте модель на тестовой выборке

Случайный лес показал лучшие результаты, его будем проверять:

In [9]:
# соединяем обучающую и валидационную выборки
features_last = pd.concat([features_train, features_valid], sort=False, axis=0)
target_last = pd.concat([target_train, target_valid], sort=False, axis=0)
# создаём модель
forest = RandomForestClassifier(random_state=23456, n_estimators=30, max_depth=8\
                                                     , min_samples_split=2, min_samples_leaf=2)
# обучаем модель
forest.fit(features_last, target_last)

result = forest.score(features_test, target_test)
print(f'accuracy на тестовой выборке {result}')

accuracy на тестовой выборке 0.807153965785381


***Вывод***

**Модель случайного леса** на тестовой выборке показала долю правильных ответов 0,807. Этот показатель нас устраивает.

## Проверка модели на адекватность

Проверим адекватнрость на фиктивных классификаторах:

In [10]:
strategies = ['most_frequent', 'stratified', 'uniform', 'constant']
# в цикле перебираем стратегии
for s in strategies:
    if s =='constant':
        dymmy = DummyClassifier(strategy = s, random_state = 23456, constant = 1)
    else:
        dymmy = DummyClassifier(strategy = s, random_state = 23456)
    dymmy.fit(features_last, target_last)
    score = dymmy.score(features_test, target_test)
    print(f'strategy = {s}, accuracy = {score}')

strategy = most_frequent, accuracy = 0.6936236391912908
strategy = stratified, accuracy = 0.5863141524105754
strategy = uniform, accuracy = 0.5147744945567652
strategy = constant, accuracy = 0.30637636080870917


# Выводы исследования

В нашем исследование даны уже хорошо предобработанные данные. Мы разбили данные на три выборки: обучающую, валидационную и тестовую.Далее построили три модели на разных алгоритмах: **дерево решений, случайный лес и логистичискую регрессию**.

Применив несколько гиперпараметров в каждой модели, мы получили, что модель ***случайного леса*** дала лучший результат по доле правельных ответов на тестовой выборке (80%). 

Процесс построения большого количества моделей в случайном лесе требует затарт на вычислительные ресурсы и времени.

Проверка модели на адекватность показала, что на случайных данных всего 69% верных ответов.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
